In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [5]:
digits=load_digits()

In [6]:
dir(digits)

['DESCR', 'data', 'images', 'target', 'target_names']

In [7]:
digits.data

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [8]:
digits.target

array([0, 1, 2, ..., 8, 9, 8])

In [9]:
data=pd.DataFrame(digits.data,digits.target)

In [10]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,13.0,15.0,10.0,15.0,5.0,0.0,0.0,3.0,15.0,2.0,0.0,11.0,8.0,0.0,0.0,4.0,12.0,0.0,0.0,8.0,8.0,0.0,0.0,5.0,8.0,0.0,0.0,9.0,8.0,0.0,0.0,4.0,11.0,0.0,1.0,12.0,7.0,0.0,0.0,2.0,14.0,5.0,10.0,12.0,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,9.0,0.0,0.0,0.0,0.0,3.0,15.0,16.0,6.0,0.0,0.0,0.0,7.0,15.0,16.0,16.0,2.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,3.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,6.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,6.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,3.0,16.0,15.0,14.0,0.0,0.0,0.0,0.0,8.0,13.0,8.0,16.0,0.0,0.0,0.0,0.0,1.0,6.0,15.0,11.0,0.0,0.0,0.0,1.0,8.0,13.0,15.0,1.0,0.0,0.0,0.0,9.0,16.0,16.0,5.0,0.0,0.0,0.0,0.0,3.0,13.0,16.0,16.0,11.0,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,13.0,6.0,15.0,4.0,0.0,0.0,0.0,2.0,1.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,2.0,15.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,8.0,0.0,0.0,0.0,8.0,4.0,5.0,14.0,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,6.0,2.0,2.0,0.0,0.0,0.0,7.0,15.0,0.0,9.0,8.0,0.0,0.0,5.0,16.0,10.0,0.0,16.0,6.0,0.0,0.0,4.0,15.0,16.0,13.0,16.0,1.0,0.0,0.0,0.0,0.0,3.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [11]:
data['target']=digits.target

In [12]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,13.0,15.0,10.0,15.0,5.0,0.0,0.0,3.0,15.0,2.0,0.0,11.0,8.0,0.0,0.0,4.0,12.0,0.0,0.0,8.0,8.0,0.0,0.0,5.0,8.0,0.0,0.0,9.0,8.0,0.0,0.0,4.0,11.0,0.0,1.0,12.0,7.0,0.0,0.0,2.0,14.0,5.0,10.0,12.0,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,9.0,0.0,0.0,0.0,0.0,3.0,15.0,16.0,6.0,0.0,0.0,0.0,7.0,15.0,16.0,16.0,2.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,3.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,6.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,6.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,3.0,16.0,15.0,14.0,0.0,0.0,0.0,0.0,8.0,13.0,8.0,16.0,0.0,0.0,0.0,0.0,1.0,6.0,15.0,11.0,0.0,0.0,0.0,1.0,8.0,13.0,15.0,1.0,0.0,0.0,0.0,9.0,16.0,16.0,5.0,0.0,0.0,0.0,0.0,3.0,13.0,16.0,16.0,11.0,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,13.0,6.0,15.0,4.0,0.0,0.0,0.0,2.0,1.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,2.0,15.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,8.0,0.0,0.0,0.0,8.0,4.0,5.0,14.0,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,6.0,2.0,2.0,0.0,0.0,0.0,7.0,15.0,0.0,9.0,8.0,0.0,0.0,5.0,16.0,10.0,0.0,16.0,6.0,0.0,0.0,4.0,15.0,16.0,13.0,16.0,1.0,0.0,0.0,0.0,0.0,3.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


In [13]:
X_train,X_test,y_train,y_test=train_test_split(data.drop(['target'],axis=1),data.target,test_size=0.2)

In [14]:
len(X_train)

1437

In [15]:
len(X_test)

360

## 1. Calculate accuracy using first method

In [16]:
model=SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.5277777777777778

##using rbf kernel got very low accuracy

In [17]:
model=SVC(kernel='linear',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9805555555555555

##using linear kernel got good accuracy

## 2. Calculate accuracy using second method

In [18]:
from sklearn.model_selection import cross_val_score

In [19]:
cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),X_train,y_train,cv=5)

array([0.46527778, 0.45138889, 0.4912892 , 0.45993031, 0.51567944])

In [20]:
cross_val_score(SVC(kernel='rbf',C=20,gamma='auto'),X_train,y_train,cv=5)

array([0.46527778, 0.45138889, 0.4912892 , 0.45993031, 0.51567944])

In [21]:
cross_val_score(SVC(kernel='linear',C=20,gamma='auto'),X_train,y_train,cv=5)

array([0.97222222, 0.97569444, 0.97560976, 0.98606272, 0.97560976])

## 3. Calculate accuracy using third method

In [22]:
kernels=['rbf','linear']
C=[1,5,10,20]
avg_scores={}
for k in kernels:
  for c in C:
    cv_scores=cross_val_score(SVC(kernel=k,C=c,gamma='auto'),X_train,y_train,cv=5)
    avg_scores[k + ' ' +str(c)]=np.average(cv_scores)
avg_scores

{'linear 1': 0.9770397793263648,
 'linear 10': 0.9770397793263648,
 'linear 20': 0.9770397793263648,
 'linear 5': 0.9770397793263648,
 'rbf 1': 0.43010307781649243,
 'rbf 10': 0.4767131242740999,
 'rbf 20': 0.4767131242740999,
 'rbf 5': 0.4767131242740999}

## 4. Calculate accuracy

In [23]:
from sklearn.model_selection import GridSearchCV

In [24]:
clf=GridSearchCV(SVC(gamma='auto'),
            {
                'C':[1,5,10,20],
             'kernel':['rbf','linear']
            },cv=5,return_train_score=False)

In [25]:
clf.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 5, 10, 20], 'kernel': ['rbf', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [26]:
clf.cv_results_

{'mean_fit_time': array([0.34250011, 0.03315997, 0.33872619, 0.03313932, 0.33914027,
        0.03328028, 0.33954101, 0.03322134]),
 'mean_score_time': array([0.04315486, 0.00949721, 0.04363165, 0.00949483, 0.04288564,
        0.0095295 , 0.04315262, 0.00952229]),
 'mean_test_score': array([0.43010308, 0.97703978, 0.47671312, 0.97703978, 0.47671312,
        0.97703978, 0.47671312, 0.97703978]),
 'param_C': masked_array(data=[1, 1, 5, 5, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 5, 'kernel': 'rbf'},
  {'C': 5, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'}

In [27]:
df1=pd.DataFrame(clf.cv_results_)

In [28]:
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.342500,0.003031,0.043155,0.000304,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.392361,0.409722,0.452962,0.414634,0.480836,0.430103,0.032173,8
1,0.033160,0.000527,0.009497,0.000160,1,linear,"{'C': 1, 'kernel': 'linear'}",0.972222,0.975694,0.975610,0.986063,0.975610,0.977040,0.004702,1
2,0.338726,0.002644,0.043632,0.001417,5,rbf,"{'C': 5, 'kernel': 'rbf'}",0.465278,0.451389,0.491289,0.459930,0.515679,0.476713,0.023599,5
3,0.033139,0.000490,0.009495,0.000122,5,linear,"{'C': 5, 'kernel': 'linear'}",0.972222,0.975694,0.975610,0.986063,0.975610,0.977040,0.004702,1
4,0.339140,0.001852,0.042886,0.000274,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.465278,0.451389,0.491289,0.459930,0.515679,0.476713,0.023599,5
5,0.033280,0.000450,0.009529,0.000141,10,linear,"{'C': 10, 'kernel': 'linear'}",0.972222,0.975694,0.975610,0.986063,0.975610,0.977040,0.004702,1
6,0.339541,0.003874,0.043153,0.000684,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.465278,0.451389,0.491289,0.459930,0.515679,0.476713,0.023599,5
7,0.033221,0.000523,0.009522,0.000165,20,linear,"{'C': 20, 'kernel': 'linear'}",0.972222,0.975694,0.975610,0.986063,0.975610,0.977040,0.004702,1


In [29]:
df1[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.430103
1,1,linear,0.977040
2,5,rbf,0.476713
3,5,linear,0.977040
4,10,rbf,0.476713
5,10,linear,0.977040
6,20,rbf,0.476713
7,20,linear,0.977040


In [30]:
clf.best_score_

0.9770397793263648

In [31]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

## 5. Calculate accuracy 

In [32]:
from sklearn.model_selection import RandomizedSearchCV

In [33]:
clr=RandomizedSearchCV(SVC(gamma='auto'),
                       {
                           'C':[1,5,10],
                        'kernel':['rbf','linear']
                       },cv=5,return_train_score=False)

In [34]:
clr.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='auto', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'C': [1, 5, 10],
                                        'kernel': ['rbf', 'linear']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [35]:
clr.best_params_

{'C': 1, 'kernel': 'linear'}

In [36]:
clr.best_score_

0.9770397793263648

##Hyperparameter Tuning

In [39]:
model_params={
    'svm':{
        'model':SVC(gamma='auto'),
        'params':{
            'C':[1,10,20],
            'kernel':['rbf','linear']
        }
        },
        'randomforest':{
            'model':RandomForestClassifier(),
            'params':{
                'n_estimators':[1,5,10]
            }
        },
        'logisticregression':{
            'model':LogisticRegression(solver='liblinear',multi_class='auto'),
            'params':{
                'C':[1,5,10]
            }
        },
        'gaussianNB':{
            'model':GaussianNB(),
            'params':{
                
            }
        },
        'multinomialNB':{
            'model':MultinomialNB(),
            'params':{
                
            }
        },
        'decisiontreeclassifier':{
            'model':DecisionTreeClassifier(),
            'params':{
                'criterion': ['gini','entropy']
            }
        }
    }
    


In [40]:
from sklearn.model_selection import GridSearchCV

scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train,y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.977040,"{'C': 1, 'kernel': 'linear'}"
1,randomforest,0.931124,{'n_estimators': 10}
2,logisticregression,0.955471,{'C': 1}
3,gaussianNB,0.821162,{}
4,multinomialNB,0.896325,{}
5,decisiontreeclassifier,0.855943,{'criterion': 'entropy'}


##Winner is svm with [C=1, kernel=linear] and accuracy 97% 